In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
import matplotlib.pyplot as plt
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from ipywidgets import interact, Select, HBox, VBox,IntSlider,SelectMultiple

# 1. Dataframe was installed from input data

In [ ]:
DF_players=pd.read_csv('/kaggle/input/nfl-big-data-bowl-2022/players.csv')
DF_ScoutingData=pd.read_csv('/kaggle/input/nfl-big-data-bowl-2022/PFFScoutingData.csv')
DF_Tracking2019=pd.read_csv('/kaggle/input/nfl-big-data-bowl-2022/tracking2019.csv')
DF_Tracking2020=pd.read_csv('/kaggle/input/nfl-big-data-bowl-2022/tracking2020.csv')
DF_Tracking2018=pd.read_csv('/kaggle/input/nfl-big-data-bowl-2022/tracking2018.csv')
DF_games=pd.read_csv('/kaggle/input/nfl-big-data-bowl-2022/games.csv')
DF_plays=pd.read_csv('/kaggle/input/nfl-big-data-bowl-2022/plays.csv')
DF_Tracking2019=DF_Tracking2019.dropna(subset=['nflId'])
DF_Tracking2020=DF_Tracking2020.dropna(subset=['nflId'])
DF_Tracking2018=DF_Tracking2018.dropna(subset=['nflId'])
DF_Tracking2020=DF_Tracking2020[DF_Tracking2020.event!='None']
DF_Tracking2018=DF_Tracking2018[DF_Tracking2018.event!='None']
DF_Tracking2019=DF_Tracking2019[DF_Tracking2019.event!='None']
DF_tracking=pd.concat([DF_Tracking2018,DF_Tracking2019,DF_Tracking2020],axis=0)
del DF_Tracking2018, DF_Tracking2020, DF_Tracking2019

# 2. investigate which kicker make some event from players_tracking data

In [ ]:
#to investigate who kicker make some events.
# plt.bar(DF_tracking.displayName.value_counts().index,DF_tracking.displayName.value_counts())
DF_Players_Tracking=pd.DataFrame(DF_tracking.groupby(['nflId', 'event']).count()['time']).reset_index().pivot(index='nflId', columns='event')
DF_Players_Tracking=DF_Players_Tracking.time
display(DF_Players_Tracking)

In [ ]:
DF_players=pd.read_csv('/kaggle/input/nfl-big-data-bowl-2022/players.csv')
DF_players=pd.merge(left=DF_players,right=DF_Players_Tracking,on='nflId')
DF_players.to_csv('/kaggle/working/Players_events.csv')

# 3.visualization

(A) Top 10 person for each events

In [ ]:
def event_scores(col1):
    plt.barh(DF_players[["displayName",col1]].sort_values(col1,ascending=False).head(10).displayName,
        DF_players[["displayName",col1]].sort_values(col1,ascending=False).head(10)[col1])
    plt.show()
# Select(description='Players Name',options=DF_players.displayName.unique())
event_name=Select(description='event Name',options=np.sort(DF_Players_Tracking.columns))
interact(event_scores, col1=event_name)

(B) average height and weight for each positon.

I found height insert error (such as 73). so I delete them...

In [ ]:
DF_players.height=DF_players[DF_players.height.str.contains('-')==True].height.str.split('-',expand=True)[0].astype(float)*0.305+DF_players[DF_players.height.str.contains('-')==True].height.str.split('-',expand=True)[1].astype(float)*0.0254

In [ ]:
# SL=DF_players.loc[DF_players.Position=='SS',:].select_dtypes(float).median()
# SL[SL!=0]
# DF_players.loc[DF_players.Position=='SS',SL[SL!=0].index]
# DF_players.Position.unique()
DF_players.groupby(['Position']).mean().drop('nflId',axis=1).to_csv('result_each_position.csv')
DF_players.groupby(['Position']).mean().drop('nflId',axis=1)

In [ ]:
SL=DF_players.loc[DF_players.Position=='SS',:].select_dtypes(float).median()
SL[SL>1]
# DF_players.loc[DF_players.Position=='SS',SL[SL!=0].index]

In [ ]:
# DF_plays=pd.merge(left=DF_plays,right=DF_players,left_on='kickerId', right_on='nflId')
# del DF_players

In [ ]:
#this data is that tracking data, plays data, and player's data was connected.
DF_plays=pd.merge(left=DF_plays,right=DF_Players_Tracking,left_on='kickerId', right_on='nflId')

In [ ]:
# #トラッキングデータから出現回数を調べる
# DF_tracking.nflId.value_counts()